In [26]:
!pip install opencv-python
!pip install pytesseract
!pip install PyPDF2
!pip install docx
!pip install nltk
!pip install rake-nltk
!pip install langdetect
!pip install sentence-transformers
!pip install pillow
!pip install PyMuPDF

In [28]:
!pip install --upgrade python-docx

Note: you may need to restart the kernel to use updated packages.


In [82]:
!pip install --upgrade pdf2image

Note: you may need to restart the kernel to use updated packages.


In [83]:
!pip install pycryptodome

  Using cached pycryptodome-3.21.0-cp36-abi3-macosx_10_9_universal2.whl.metadata (3.4 kB)
Using cached pycryptodome-3.21.0-cp36-abi3-macosx_10_9_universal2.whl (2.5 MB)


In [78]:
for r,s,f in os.walk("/"):
    for i in f:
        if "tesseract" in i:
            print(os.path.join(r,i))
!chmod 755 "/opt/homebrew/Cellar/tesseract/5.5.0/bin/tesseract"
pytesseract.pytesseract.tesseract_cmd = r"/opt/homebrew/Cellar/tesseract/5.5.0/bin/tesseract"

In [51]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [36]:
import os
import cv2
import json
import re
import pytesseract
import fitz
import unicodedata
import uuid
import PyPDF2

from docx import Document
from PIL import Image, ImageOps, ImageEnhance
import numpy as np
import nltk
from nltk.corpus import stopwords
from rake_nltk import Rake
from collections import Counter
from langdetect import detect
from typing import Dict, Any, List
from sentence_transformers import SentenceTransformer
from datetime import datetime

### Обработка документа

### 0. Скачиваем стоп-слова при первом запуске

In [34]:
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

nltk.download('punkt')

RUSSIAN_STOPWORDS = set(stopwords.words('russian'))

[nltk_data] Downloading package punkt to /Users/benristar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1. Извлечение метаданных

In [ ]:
def parse_pdf_date(date_str: str) -> datetime:
    """
    Парсит дату из PDF метаданных в различных форматах.
    
    Args:
        date_str: Строка с датой в формате PDF (например, "D:20230512163128+03'00'")
    
    Returns:
        datetime: Объект datetime с датой создания
    """
    # Убираем префикс 'D:' если есть
    date_str = date_str.replace('D:', '')
    
    # Извлекаем основную часть даты (YYYYMMDDHHMMSS)
    date_pattern = r'(\d{14}|\d{12}|\d{8})'
    match = re.search(date_pattern, date_str)
    
    if not match:
        raise ValueError(f"Неверный формат даты: {date_str}")
        
    date_str = match.group(1)
    
    # Обрабатываем разные длины дат
    if len(date_str) == 14:  # YYYYMMDDHHMMSS
        return datetime.strptime(date_str, '%Y%m%d%H%M%S')
    elif len(date_str) == 12:  # YYYYMMDDHHMM
        return datetime.strptime(date_str, '%Y%m%d%H%M')
    else:  # YYYYMMDD
        return datetime.strptime(date_str, '%Y%m%d')

def extract_metadata_pdf(pdf_path: str) -> Dict[str, any]:
    """
    Извлекает автора и дату создания из PDF-документа.
    Возвращает словарь с ключами 'author' и 'created_date'.
    """
    try:
        metadata = {'author': None, 'created_date': None}
        
        with open(pdf_path, 'rb') as f:
            pdf = PyPDF2.PdfReader(f)
            
            # Проверяем, зашифрован ли документ
            if pdf.is_encrypted:
                try:
                    pdf.decrypt('')
                except:
                    print(f"PDF файл {pdf_path} защищен паролем")
                    return metadata
            
            if pdf.metadata:
                # Извлекаем автора
                author = pdf.metadata.get('/Author', None)
                if author:
                    metadata['author'] = str(author)

                # Извлекаем дату создания
                created_date = pdf.metadata.get('/CreationDate', None)
                if created_date:
                    try:
                        date_obj = parse_pdf_date(str(created_date))
                        metadata['created_date'] = date_obj.strftime('%Y-%m-%d')
                    except Exception as e:
                        print(f"Не удалось преобразовать дату создания: {created_date}. Ошибка: {str(e)}")
                        
                        # Пробуем альтернативное поле даты, если есть
                        mod_date = pdf.metadata.get('/ModDate', None)
                        if mod_date:
                            try:
                                date_obj = parse_pdf_date(str(mod_date))
                                metadata['created_date'] = date_obj.strftime('%Y-%m-%d')
                            except:
                                print(f"Не удалось преобразовать альтернативную дату: {mod_date}")
                
        return metadata
    except Exception as e:
        print(f"Не удалось извлечь метаданные из PDF: {e}")
        return {'author': None, 'created_date': None}


def extract_metadata_word(docx_path: str) -> Dict[str, any]:
    """
    Извлекает автора и дату создания из Word-документа.
    Возвращает словарь с ключами 'author' и 'created_date'.
    """
    try:
        metadata = {'author': None, 'created_date': None}
        
        doc = Document(docx_path)
        core_props = doc.core_properties
        
        # Извлекаем автора
        author = core_props.author
        if author:
            metadata['author'] = str(author)
            
        # Извлекаем дату создания
        created_date = core_props.created
        if created_date:
            metadata['created_date'] = created_date.strftime('%Y-%m-%d')
            
        return metadata
    except Exception as e:
        print(f"Не удалось извлечь метаданные из Word: {e}")
        return {'author': None, 'created_date': None}





def preprocess_text(text: str) -> List[str]:
    """
    Очищает текст: удаляет пунктуацию, разрезает на слова и приводит к нижнему регистру.
    """
    # Убираем пунктуацию и лишние символы
    text = re.sub(r'[^\w\s]', '', text)
    # Разбиваем на слова и приводим к нижнему регистру
    words = text.lower().split()
    return words


def generate_tags_multilang(text: str, num_tags: int = 5) -> List[str]:
    """
    Генерирует ключевые слова на основе текста, поддерживает русский и английский языки.
    
    Args:
        text (str): Текст, из которого извлекаем ключевые слова.
        num_tags (int): Количество выводимых ключевых слов.

    Returns:
        List[str]: Список ключевых слов.
    """
    try:
        # Определяем язык текста
        lang = detect(text)
        print(f"Определён язык текста: {lang}")

        # Выбираем стоп-слова в зависимости от языка
        if lang == 'ru':
            stop_words = RUSSIAN_STOPWORDS
        else:
            stop_words = ENGLISH_STOPWORDS

        # Токенизация текста
        words = preprocess_text(text)

        # Убираем стоп-слова
        meaningful_words = [word for word in words if word not in stop_words]

        # Подсчитываем частоту слов
        word_counts = Counter(meaningful_words)

        # Извлекаем num_tags самых частых слов
        tags = [word for word, _ in word_counts.most_common(num_tags)]
        
        return tags

    except Exception as e:
        print(f"Ошибка формирования тегов: {e}")
        return []

### 2. Извлечение текста

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """Извлекает текст из PDF-документа, включая зашифрованные файлы."""
    try:
        text = ""
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            
            # Проверяем, зашифрован ли документ
            if reader.is_encrypted:
                try:
                    # Пробуем расшифровать без пароля
                    reader.decrypt('')
                except:
                    print(f"PDF файл {pdf_path} защищен паролем")
                    return ""
            
            # Извлекаем текст
            for page in reader.pages:
                try:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text
                except Exception as e:
                    print(f"Ошибка при извлечении текста со страницы: {e}")
                    continue
                    
        return text.strip()
    except Exception as e:
        print(f"Ошибка извлечения текста из PDF {pdf_path}: {e}")
        return ""

def extract_text_from_word(word_path: str) -> str:
    """Извлекает текст из Word-документа."""
    try:
        doc = Document(word_path)
        return "\n".join(paragraph.text for paragraph in doc.paragraphs).strip()
    except Exception as e:
        print(f"Ошибка извлечения текста из Word {word_path}: {e}")
        return ""

### 3. Извлечение изображений

In [55]:
def create_document_folder(document_id: str) -> str:
    """Создает папку для хранения изображений документа."""
    try:
        folder_path = f"data/{document_id}"
        os.makedirs(folder_path, exist_ok=True)
        return folder_path
    except Exception as e:
        print(f"Ошибка создания папки: {e}")
        return ""

def preprocess_image(image_path: str) -> Image:

    """Комплексная предобработка изображения."""
    
    try:
    
        # Загрузка изображения
    
        image = Image.open(image_path)

        # Конвертация в оттенки серого
    
        gray_image = ImageOps.grayscale(image)
    
        # Увеличение резкости
    
        enhancer = ImageEnhance.Sharpness(gray_image)
    
        final_image = enhancer.enhance(2.0)

        return final_image
    
    except Exception as e:
    
        print(f"Ошибка предобработки изображения: {e}")
    
        return Image.open(image_path)


def has_text_in_image(image_path: str) -> bool:
    """Проверяет, содержит ли изображение текст."""
    try:
        # Предобработка изображения
        processed_image = preprocess_image(image_path)
        # Конфигурация OCR
        config = r'--oem 3 --psm 3'
        # Распознавание текста с улучшенной конфигурацией
        text = pytesseract.image_to_string(processed_image, lang="rus+eng", config=config)
        return bool(text.strip())
    except Exception as e:
        print(f"Ошибка проверки текста в изображении {image_path}: {e}")
        return False

def extract_images_from_pdf(pdf_path: str, folder_path: str) -> List[str]:
    """Извлекает изображения из PDF-документа и сохраняет только те, что содержат текст."""
    images = []
    try:
        doc = fitz.open(pdf_path)
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            for img_index, img in enumerate(page.get_images(full=True)):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]
                # Временно сохраняем изображение для проверки
                temp_path = f"{folder_path}/temp_image.{image_ext}"
                with open(temp_path, "wb") as f:
                    f.write(image_bytes)
                # Проверяем наличие текста
                if has_text_in_image(temp_path):
                    # Если текст есть, сохраняем изображение с нормальным именем
                    image_path = f"{folder_path}/image_page{page_num + 1}_img{img_index + 1}.{image_ext}"
                    os.rename(temp_path, image_path)
                    images.append(image_path)
                else:
                    # Если текста нет, удаляем временный файл
                    os.remove(temp_path)
    except Exception as e:
        print(f"Ошибка извлечения изображений из PDF {pdf_path}: {e}")
    return images

def extract_images_from_word(word_path: str, folder_path: str) -> List[str]:
    """Извлекает изображения из Word-документа и сохраняет только те, что содержат текст."""
    images = []
    try:
        doc = Document(word_path)
        for index, rel in enumerate(doc.part.rels.values()):
            if "image" in rel.target_ref:
                image_data = rel.target_part.blob
                # Временно сохраняем изображение для проверки
                temp_path = f"{folder_path}/temp_image_{index}.png"
                with open(temp_path, "wb") as f:
                    f.write(image_data)
                # Проверяем наличие текста
                if has_text_in_image(temp_path):
                    # Если текст есть, сохраняем изображение с нормальным именем
                    image_path = f"{folder_path}/image_word_{len(images) + 1}.png"
                    os.rename(temp_path, image_path)
                    images.append(image_path)
                else:
                    # Если текста нет, удаляем временный файл
                    os.remove(temp_path)
    except Exception as e:
        print(f"Ошибка извлечения изображений из Word {word_path}: {e}")
    return images


### 4. OCR для изображений

In [ ]:
def clean_ocr_text(text: str) -> str:
    """Очищает распознанный текст от мусора и нормализует его."""
    try:
        # Удаляем специальные символы, оставляя буквы, цифры, точки и запятые
        cleaned = re.sub(r'[^a-zA-Zа-яА-Я0-9.,\s]', '', text)
        
        # Заменяем множественные пробелы на один
        cleaned = re.sub(r'\s+', ' ', cleaned)
        
        # Удаляем пробелы перед знаками препинания
        cleaned = re.sub(r'\s+([.,])', r'\1', cleaned)
        
        # Удаляем отдельно стоящие буквы (вероятные ошибки распознавания)
        cleaned = re.sub(r'\s+[a-zA-Zа-яА-Я]\s+', ' ', cleaned)
        
        # Удаляем строки, содержащие менее 2 символов
        lines = [line.strip() for line in cleaned.split('\n') if len(line.strip()) > 2]
        
        return '\n'.join(lines).strip()
    except Exception as e:
        print(f"Ошибка очистки текста: {e}")
        return text

def extract_text_from_images(image_paths: List[str]) -> List[str]:
    """Применяет OCR ко всем изображениям из списка с улучшенной обработкой текста."""
    texts = []
    
    for image_path in image_paths:
        try:
            # Предобработка изображения
            processed_image = preprocess_image(image_path)
            
            # Применяем OCR с поддержкой обоих языков
            config = r'--oem 3 --psm 3 -c preserve_interword_spaces=1'
            
            # Используем оба языка одновременно
            ocr_text = pytesseract.image_to_string(
                processed_image,
                lang="rus+eng",
                config=config
            )
            
            # Очищаем и нормализуем текст
            cleaned_text = clean_ocr_text(ocr_text)
            
            # Дополнительная проверка на качество распознавания
            if len(cleaned_text) > 0:
                # Разбиваем на строки и фильтруем пустые
                lines = [line.strip() for line in cleaned_text.split('\n') if line.strip()]
                # Объединяем обратно в текст
                final_text = '\n'.join(lines)
                texts.append(final_text)
            else:
                texts.append("")
                
        except Exception as e:
            print(f"Ошибка OCR для {image_path}: {e}")
            texts.append("")
            
    return texts

### 5. Очистка текста

In [ ]:
def decode_unicode_sequence(text: str) -> str:
    """Декодирует строки Unicode в читаемый текст."""
    # Найдем все /uniXXXX и преобразуем их в символы Unicode
    decoded_text = re.sub(
        r"/uni([0-9A-Fa-f]{4})",  # Паттерн для /uniXXXX
        lambda x: chr(int(x.group(1), 16)),  # Преобразование кодов в Unicode-символы
        text
    )
    return decoded_text

def clean_text(text: str) -> str:
    """Очищает текст от лишних пробелов, некорректных символов и декодирует Unicode."""
    try:
        # Декодируем Unicode-последовательности
        text = decode_unicode_sequence(text)

        # Удаляем все специальные символы, оставляя буквы, цифры, точки и запятые
        text = re.sub(r'[^a-zA-Zа-яА-Я0-9.,\s]', '', text)

        # Удаляем лишние пробелы
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Возвращаем очищенный текст
        return text
    except Exception as e:
        print(f"Ошибка очистки текста: {e}")
        return text

### 6. Генерация ID

In [ ]:
def generate_document_id() -> str:
    """
    Генерация уникального ID документа с использованием uuid4.
    """
    return str(uuid.uuid4())

### 7. Векторизация

In [ ]:
# Инициализируем модель для векторизации (желательно сделать это один раз при запуске приложения)
vectorizer_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
def vectorize_text(text: str) -> list:
    """Преобразует текст в dense vector."""
    try:
        dense_vector = vectorizer_model.encode(text, convert_to_numpy=True)
        return dense_vector.tolist()
    except Exception as e:
        print(f"Ошибка векторизации текста: {e}")
        return []

### 8. Процессинг документа

In [80]:
def process_document(file_path: str) -> Dict[str, Any]:
    """Общий процесс обработки документа (PDF или Word)."""
    ext = os.path.splitext(file_path)[-1].lower()
    result = {}

    # Получаем название файла без расширения и создаем ID документа
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    document_id = generate_document_id()    

    # Создаем папку для изображений документа
    folder_path = create_document_folder(document_id)

    # Извлекаем текст, метаданные и изображения
    if ext == ".pdf":
        text = extract_text_from_pdf(file_path)
        metadata = extract_metadata_pdf(file_path)
        images = extract_images_from_pdf(file_path, folder_path)
    elif ext == ".docx":
        text = extract_text_from_word(file_path)
        metadata = extract_metadata_word(file_path)
        images = extract_images_from_word(file_path, folder_path)
    else:
        print(f"Формат файла {ext} не поддерживается.")
        return {}

    # Применяем OCR для изображений
    ocr_texts = extract_text_from_images(images)

    # Очищаем текст
    cleaned_text = clean_text(text + " " + " ".join(ocr_texts))

    # Векторизуем очищенный текст
    text_vector = vectorize_text(cleaned_text)

    # Теги



    # Собираем итоговый результат
    result["document_id"] = document_id
    result["title"] = file_name  # Название файла без расширения
    result["text_content"] = cleaned_text
    result["text_content_vector"] = text_vector  # Добавляем векторное представление текста
    metadata["tags"] = generate_tags_multilang(cleaned_text, num_tags=10)  # Сгенерированные ключевые слова для текста
    result["metadata"] = metadata  # Все метаданные, включая теги


    # Формируем информацию о каждом изображении
    result["images"] = []
    for idx, (img_path, ocr_text) in enumerate(zip(images, ocr_texts), start=1):
        image_info = {
            "image_id": f"img_{idx}",
            "ocr_text": ocr_text,
            "position": f"Page {idx}",
            "image_path": img_path
        }
        result["images"].append(image_info)

    return result


# Тестирование обработки одного файла
if __name__ == "__main__":
    file_path = '/Users/benristar/Development/Проекты/NornickelHack2024/dataset/Постановление_Правительства_РФ_от_16_02_2008_N_87_О_составе_разделов.docx' 
    output = process_document(file_path)
    print(json.dumps(output, indent=4, ensure_ascii=False))

Определён язык текста: ru
{
    "document_id": "37ad125a-5b6f-42d2-b78a-f80582ec802e",
    "title": "Постановление_Правительства_РФ_от_16_02_2008_N_87_О_составе_разделов",
    "text_content": "ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ ПОСТАНОВЛЕНИЕ от 16 февраля 2008 г. N 87 О СОСТАВЕ РАЗДЕЛОВ ПРОЕКТНОЙ ДОКУМЕНТАЦИИ И ТРЕБОВАНИЯХ К ИХ СОДЕРЖАНИЮ в ред. Постановлений Правительства РФ от 18.05.2009 N 427, от 21.12.2009 N 1044, от 13.04.2010 N 235, от 07.12.2010 N 1006, от 15.02.2011 N 73, от 25.06.2012 N 628, от 02.08.2012 N 788, от 22.04.2013 N 360, от 30.04.2013 N 382, от 08.08.2013 N 679, от 26.03.2014 N 230, от 10.12.2014 N 1346, от 28.07.2015 N 767, от 27.10.2015 N 1147, от 23.01.2016 N 29, от 12.11.2016 N 1159, от 28.01.2017 N 95, от 28.04.2017 N 506, от 12.05.2017 N 563, от 07.07.2017 N 806, от 08.09.2017 N 1081, от 13.12.2017 N 1541, от 15.03.2018 N 257, от 21.04.2018 N 479, от 17.09.2018 N 1096, от 06.07.2019 N 864, от 28.04.2020 N 598, от 01.10.2020 N 1590, от 21.12.2020 N 2184, от 09

In [84]:

def process_all_files_in_folder(folder_path: str, output_file: str):
    """Обрабатывает все файлы в папке и сохраняет результаты в один JSON-файл."""
    results = []
    supported_extensions = {".pdf", ".docx"}

    # Перебираем все файлы в папке
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Проверяем, является ли файл документом (PDF/Word) по его расширению
        if os.path.isfile(file_path) and file_name.lower().endswith(tuple(supported_extensions)):
            print(f"Обрабатывается файл: {file_name}")
            try:
                # Обрабатываем документ
                result = process_document(file_path)
                results.append(result)
            except Exception as e:
                print(f"Ошибка обработки файла {file_name}: {e}")

    # Сохраняем все результаты в итоговый JSON-файл
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4, ensure_ascii=False)
        print(f"Итоговый JSON сохранён в файл: {output_file}")


if __name__ == "__main__":
    folder_path = '/Users/benristar/Development/Проекты/NornickelHack2024/dataset'  # Укажите путь к папке с документами
    output_file = 'output.json'  # Название файла, куда будет сохранён результат
    process_all_files_in_folder(folder_path, output_file)


Обрабатывается файл: responsible_supply_chain_report_rus.pdf
Ошибка предобработки изображения: broken data stream when reading image file
Ошибка проверки текста в изображении data/aa90cf0c-4dc2-4f03-afca-ee812c6d1c63/temp_image.jpx: broken data stream when reading image file
Определён язык текста: ru
Обрабатывается файл: nn_cso_2023_rus.pdf
Определён язык текста: ru
Обрабатывается файл: NN_AR_2021_Book_RUS_26.09.22.pdf
Определён язык текста: ru
Обрабатывается файл: KPMG_Global Metals & Mining_2024 (48 pgs).pdf
Ошибка извлечения текста из PDF /Users/benristar/Development/Проекты/NornickelHack2024/dataset/KPMG_Global Metals & Mining_2024 (48 pgs).pdf: PyCryptodome is required for AES algorithm
Не удалось извлечь метаданные из PDF: PyCryptodome is required for AES algorithm
Ошибка формирования тегов: No features in text.
Обрабатывается файл: NN_CSO2021_RUS_03.03.2023.pdf
Определён язык текста: ru
Обрабатывается файл: СП_496_1325800_2020_Основания_и_фундаменты_зданий_и_сооружений.docx
Оп

Overwriting cache for 0 5388


Определён язык текста: en
Ошибка формирования тегов: name 'ENGLISH_STOPWORDS' is not defined
Обрабатывается файл: 2_5366183129474161642.pdf


Overwriting cache for 0 5388


Определён язык текста: ru
Обрабатывается файл: nn_climate_change_report_rus.pdf
Определён язык текста: ru
Обрабатывается файл: Норникель_Внутрення_цена_на_углерод.pdf
Определён язык текста: ru
Обрабатывается файл: НЛМК 2024.pdf
Определён язык текста: ru
Обрабатывается файл: ММК 2024.pdf
Определён язык текста: ru
Обрабатывается файл: 2022_Annual_Report_of_PJSC_MMC_Norilsk_Nickel_rus.pdf
Определён язык текста: ru
Обрабатывается файл: Постановление_Правительства_РФ_от_16_02_2008_N_87_О_составе_разделов.docx
Определён язык текста: ru
Обрабатывается файл: Kept_Обзор_цен_в_металлургической_отрасли_2К2024.pdf
Ошибка извлечения текста из PDF /Users/benristar/Development/Проекты/NornickelHack2024/dataset/Kept_Обзор_цен_в_металлургической_отрасли_2К2024.pdf: PyCryptodome is required for AES algorithm
Не удалось извлечь метаданные из PDF: PyCryptodome is required for AES algorithm
Ошибка формирования тегов: No features in text.
Итоговый JSON сохранён в файл: output.json
